In [1]:
import os
import re

In [ ]:
from lingtrain_aligner import preprocessor, splitter, aligner, resolver, helper, reader, vis_helper

In [3]:
aligner.align_texts.__doc__

In [4]:
with open('JP_eng.txt', 'r') as f:
    data = f.read()

with open('JP_eng.txt', 'w') as f:
    f.write(re.sub(r'”|“', '', data))

In [5]:
source = open('JP_de.txt','r',encoding='utf-8')
source_line=source.readlines()
target = open('JP_eng.txt','r',encoding='utf-8')
target_line=target.readlines()

In [6]:
source_prepared = preprocessor.mark_paragraphs(source_line)
target_prepared = preprocessor.mark_paragraphs(target_line)
splitted_from = splitter.split_by_sentences_wrapper(source_prepared, 'de', leave_marks=True)
splitted_to = splitter.split_by_sentences_wrapper(target_prepared, 'eng', leave_marks=True)

In [7]:
print(len(splitted_to))
print(len(splitted_from))

5805
5898


In [8]:
#создаем базу данных

db_path='result_db.db'
if os.path.isfile(db_path):
    os.unlink(db_path)

aligner.fill_db(db_path,'de', 'eng', splitted_from, splitted_to)

In [ ]:
#models = ["sentence_transformer_multilingual", "sentence_transformer_multilingual_labse"]
model_name = "sentence_transformer_multilingual"
batch_ids = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59]

db=aligner.align_db(db_path, \
                model_name, \
                batch_size=100, \
                window=40, \
                batch_ids=batch_ids, \
                save_pic=False,
                embed_batch_size=10, \
                normalize_embeddings=True, \
                show_progress_bar=True
                )

In [ ]:
vis_helper.visualize_alignment_by_db(db_path, output_path="alignment_vis.png", \
                                    lang_name_from="de", \
                                    lang_name_to="eng", \
                                    batch_size=5800, \
                                    size=(800,800), \
                                    plt_show=True)

In [ ]:
conflicts_to_solve, rest = resolver.get_all_conflicts(db_path, min_chain_length=2, max_conflicts_len=6, batch_id=-1)

resolver.get_statistics(conflicts_to_solve)
resolver.get_statistics(rest)

In [ ]:
resolver.show_conflict(db_path, conflicts_to_solve[10])

In [ ]:
steps = 3
batch_id = -1 

for i in range(steps):
    conflicts, rest = resolver.get_all_conflicts(db_path, min_chain_length=2+i, max_conflicts_len=6*(i+1), batch_id=batch_id)

    resolver.resolve_all_conflicts(db_path, conflicts, model_name, show_logs=False)

    vis_helper.visualize_alignment_by_db(db_path, output_path="img_test1.png", batch_size=5800, size=(800,800), plt_show=True)

    if len(rest) == 0:
        break

In [20]:
from lingtrain_aligner import reader

paragraphs_from, paragraphs_to, meta = reader.get_paragraphs(db_path, direction="from")
reader.create_book(paragraphs_from, 
                    paragraphs_to,
                    meta,
                    output_path = "pr&pr.html",
                    template="pastel_fill",
                    )

In [22]:
from bs4 import BeautifulSoup

with open("pr&pr.html", "r", encoding="utf-8") as f:
    html_content = f.read()        
    soup = BeautifulSoup(html_content, 'html.parser')
    new_img_tag = soup.new_tag('img', src="pp.jpg")
    style_tag = soup.new_tag('style')
    style_content = """
        body {
            margin: 0;
            display: flex;
            flex-direction: column; 
            align-items: center;
        }
        img {
            max-width: 100%; 
            height: auto; 
            margin-top: 20px;
        }
    """
    style_tag.string = style_content
    target_element = soup.select_one("head")
    target_element.append(style_tag)
    target_body = soup.select_one("body")
    target_body.insert(0, new_img_tag) 

with open("pr&pr.html", "w", encoding="utf-8") as f:
    f.write(soup.prettify(formatter="html"))